# **Data Collectors and Landing Zone**

### *Required packages:*

In [ ]:
#!pip install opendatasets
#!pip install kaggle
#!pip install openpyxl

In [ ]:
import numpy as np
import pandas as pd
import requests
import os
import opendatasets as od
import csv
import openpyxl

In [ ]:
!mkdir ~/.kaggle

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [ ]:
!cp /kaggle_fitxer.json /kaggle/kaggle.json

In [ ]:
!cd /kaggle && ls

input  kaggle.json


### *Code:*

#### Downloading datasets

In [ ]:
# URL del dataset UNECE
unece_db = pd.read_json('https://query.data.world/s/qd6kjqguuvb573mffbpfysgpudi2pv?dws=00000')
unece_db.to_csv('unece_db.csv', index=False)
!mv /content/unece_db.csv /content/data/unece_db.csv

In [ ]:
# API Kaggle Mental Health
!kaggle datasets download  -d 'thedevastator/uncover-global-trends-in-mental-health-disorder'
#!sudo apt-get install unzip
!unzip uncover-global-trends-in-mental-health-disorder.zip -d data/

uncover-global-trends-in-mental-health-disorder.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  uncover-global-trends-in-mental-health-disorder.zip
replace data/Mental health Depression disorder Data.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
# API United States Census
variables = "NAME","YR","AREA_KM2","CBR","CDR","DEATHS","E0","GRR","IMR","MEDAGE","MR0_4","POP_DENS","genc standard countries and areas"
url_base = "https://api.census.gov/data/timeseries/idb/5year?get=NAME,YR,AREA_KM2,CBR,CDR,DEATHS,E0,GRR,IMR,MEDAGE,MR0_4,POP_DENS&for=genc%20standard%20countries%20and%20areas:*".format(variables)

# Llista per guardar les dades
data = []

# Fem una sol·licitud a l'API per agafar dades des de l'any 1990 fins al 2019
for year in range(1990, 2019):
    url = "{}&time={}".format(url_base, year)
    response = requests.get(url)
    if response.status_code == 200:
        data.extend(response.json()[1:])

# Guardar los datos en un archivo CSV
csv_filename = "dades_demogr_census.csv"
with open(csv_filename, "w", newline="") as csv_file:
    csv_writer = csv.writer(csv_file)
    csv_writer.writerow(["NAME","YR","AREA_KM2","CBR","CDR","DEATHS","E0","GRR","IMR","MEDAGE","MR0_4","POP_DENS","genc standard countries and areas", "code"])
    csv_writer.writerows(data)

print("Les dades s'han guardat a", csv_filename)

!mv /content/dades_demogr_census.csv /content/data/dades_demogr_census.csv


Les dades s'han guardat a dades_demogr_census.csv


#### Storing files in a suitable format (Parquet)


In [58]:
# Unece_db
unece_db.to_parquet('/content/data/unece_db.parquet')

# Kaggle db
kaggle_db = pd.read_csv('/content/data/Mental health Depression disorder Data.csv', decimal=',')
kaggle_db.to_parquet('/content/data/kaggle_db.parquet')

# United State Census db
us_census_db = pd.read_csv('/content/data/dades_demogr_census.csv')
us_census_db.to_parquet('/content/data/us_census_db.parquet')

<ipython-input-58-a336a83c75fc>:5: DtypeWarning: Columns (7,8,9,10) have mixed types. Specify dtype option on import or set low_memory=False.
  kaggle_db = pd.read_csv('/content/data/Mental health Depression disorder Data.csv', decimal=',')
